**Add Measures to make an Optimization OpenStudio Workflow (OSW)**

The OpenStudio SDK provides a tool for programmatically making changes to an OpenStudio Model. By using the OpenStudio Workflow (OSW), you can define a series of measures that make these changes in a specified order. We will add measures from the calibration-measures gem to the baseline model workflow described in BaselineModel.ipynb. We will also change measure arguments by modifying the measure's properties in the OSW file.

To begin, load the 'openstudio' Ruby bindings to the SDK as well as 'fileutils' for basic file/directory manipulation.

In [1]:
require 'fileutils'
require 'openstudio'

true

create project directory called 'Optimization' and change the working directory to match.

In [2]:
project_dir = 'Optimization'
#create project folder
FileUtils.mkdir_p("#{project_dir}") unless Dir.exist?("#{project_dir}")

#change working directory
Dir.chdir "#{project_dir}"

0

Create the example model and save in the 'seeds' folder

In [4]:
# Create the Example Model
model = OpenStudio::Model.exampleModel()

#create directory for the seed model
FileUtils.mkdir_p("seeds") unless Dir.exist?("seeds")
# Save the model to an OSM file
osm_path = OpenStudio::Path.new(File.expand_path("seeds/example_model.osm"))
model.save(osm_path, true)


true

Get the weather file from previous download or redownload and put in the weather directory.  Set the weather file into the Model and do some basic validation on it. 

In [5]:
#get weather file
require 'open-uri'

# URL of the file to download
url = 'https://github.com/NREL/OpenStudio/raw/develop/resources/utilities/Filetypes/'

# Local file name
weather_file_name = 'USA_CO_Golden-NREL.724666_TMY3.epw'
weather_file_path = "weather/" + weather_file_name

#create directory for the weather file
FileUtils.mkdir_p("weather") unless Dir.exist?("weather")

if !File.exist?(weather_file_path)
    puts "Download the file #{weather_file_name}"
    URI.open(url + weather_file_name) do |file|
      File.write(weather_file_path, file.read)
    end
else
    puts 'weather file exists locally'
end

#set the weather file in the OSM
#use absolute path
epw_file = OpenStudio::EpwFile.load(OpenStudio::Path.new(File.expand_path(weather_file_path)), true)
OpenStudio::Model::WeatherFile::setWeatherFile(model, epw_file.get)
weatherFile = model.getWeatherFile
# validate the weather file
puts weatherFile.checksum.get == 'BDF687C1'
puts weatherFile.country == 'USA'
puts weatherFile.stateProvinceRegion == 'CO'

Download the file USA_CO_Golden-NREL.724666_TMY3.epw
true
true
true


use the 'Open3' library to make a system call to the OpenStudio command-line interface (CLI) and check its version. The CLI path is stored in the 'cli' variable and the system call is made using the 'capture3' method of the 'Open3' module, which captures the standard output, standard error, and exit status of the system command. The result of the system call is checked using the 'status.success?' method, and if it is successful, the standard output is printed to the console. If it fails, the standard error is printed instead.

In [6]:
require 'open3'

#get path to OpenStudio CLI
cli = OpenStudio.getOpenStudioCLI.to_s

# Make a system call to get version
stdout, stderr, status = Open3.capture3("#{cli} --version")

# Check the result
if status.success?
  puts "Command executed successfully"
  puts stdout
else
  puts "Command failed"
  puts stderr
end

Command executed successfully
3.5.0+7b14ce1588


require the "openstudio-calibration" and "openstudio-common-measures" ruby gems. These gems contain pre-written scripts (OpenStudio Measures) that can make programmatic changes to an OpenStudio model, that we want to use in our project. Use the "Gem::Specification" method or the "Gem.find_files" method to find the local path to the /lib folder of the gem which is where the Measures are located. These paths are then stored in the variables "calibration_measures_dir" and "common_measures_dir".

In [7]:
require 'openstudio-calibration'
require 'openstudio-common-measures'

#find path to the Measures
puts Pathname.new(Gem.find_files('openstudio-common-measures.rb').first).dirname
# -or
#find path to Measures using gem::spec
common_measures_dir = Gem::Specification.find_by_name("openstudio-common-measures").gem_dir
calibration_measures_dir = Gem::Specification.find_by_name("openstudio-calibration").gem_dir

C:/Ruby27-x64/lib/ruby/gems/2.7.0/gems/openstudio-common-measures-0.7.0/lib


"C:/Ruby27-x64/lib/ruby/gems/2.7.0/gems/openstudio-calibration-0.7.0"

copy the OpenStudio Measures from the openstudio-common-measures and openstudio-calibration gems into a local project directory called "measures". The purpose of this is to make these scripts available for use in the current project. The measures copied include "openstudio_results", "ExteriorWallThermalPropertiesPercentChange", "RoofThermalPropertiesPercentChange", "CoilHeatingGasPercentChange", "CoilCoolingDXSingleSpeedPercentChange", "FansPercentChange" and "GeneralCalibrationMeasurePercentChange".

**NOTE** the CoilCoolingDXSingleSpeedPercentChange Measure needs calibration gem version >= 0.8.0 due to an OS API change.

In [8]:
#make measure directories for the project

FileUtils.mkdir_p("measures") unless Dir.exist?("measures")

#copy measure from gem to local project directory
FileUtils.cp_r("#{common_measures_dir}/lib/measures/openstudio_results", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/ExteriorWallThermalPropertiesPercentChange", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/RoofThermalPropertiesPercentChange", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/CoilHeatingGasPercentChange", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/CoilCoolingDXSingleSpeedPercentChange", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/FansPercentChange", "measures")
FileUtils.cp_r("#{calibration_measures_dir}/lib/measures/GeneralCalibrationMeasurePercentChange", "measures")

The OSW file assumes that the associated files and directories are in the same location.

  /calibration_workflow.osw  
  /measures  
  /seeds  
  /weather   


Create the OSW as before and add the Measures to the ModelMeasure step.

In [9]:
# Create a new OSW
osw = OpenStudio::WorkflowJSON.new

# Define the reporting measure steps
measure_steps = []

#add Utility Bill object to model from external JSON file
os_results = OpenStudio::MeasureStep.new("measures/ExteriorWallThermalPropertiesPercentChange")
os_results.setArgument("r_value_perc_change",10.0)
measure_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/RoofThermalPropertiesPercentChange")
os_results.setArgument("r_value_perc_change",10.0)
measure_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/CoilHeatingGasPercentChange")
os_results.setArgument("coil","*All Gas Heating Coils*")
os_results.setArgument("coil_efficiency_perc_change",10.0)
measure_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/CoilCoolingDXSingleSpeedPercentChange")
os_results.setArgument("coil","*All DX Cooling Coils*")
os_results.setArgument("rated_cop_perc_change",10.0)
measure_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/FansPercentChange")
os_results.setArgument("fan","*All Fans*")
os_results.setArgument("fan_efficiency_perc_change",10.0)
measure_steps << os_results

os_results = OpenStudio::MeasureStep.new("measures/GeneralCalibrationMeasurePercentChange")
os_results.setArgument("space_type","*All SpaceTypes*")
os_results.setArgument("space","*All Spaces*")
os_results.setArgument("lights_perc_change",10.0)
os_results.setArgument("ElectricEquipment_perc_change",10.0)
measure_steps << os_results

# Set the measure steps in the OSW
measure_type = OpenStudio::MeasureType.new("ModelMeasure")
osw.setMeasureSteps(measure_type, measure_steps)

# Define the reporting measure steps
reporting_steps = []

#add openstudio_results to the reporting_steps array
os_results = OpenStudio::MeasureStep.new("measures/openstudio_results")
reporting_steps << os_results

# Set the measure steps in the OSW
measure_type = OpenStudio::MeasureType.new("ReportingMeasure")
osw.setMeasureSteps(measure_type, reporting_steps)

true

Add the example_model OSM we generated earlier as the 'seed' model and set the weatherfile in the OSW. Also, save the OSW as 'optimization_workflow.osw' by creating a full, absolute path to the file name and use the .saveAs() method.

In [10]:
#add seed / weatherfile to OSW
osw.setSeedFile(osm_path)
osw.setWeatherFile(weather_file_path)
#save file locally
osw_path = OpenStudio::Path.new(File.expand_path("optimization_workflow.osw"))
osw.saveAs(osw_path)

true

Since this is in Ruby and the CLI is an executable, to run the workflow using the CLI, we will use the Open3.capture3 method to make a System Call to the CLI, which captures the standard output, standard error, and status code of the CLI command. The CLI command runs the workflow, specifying the workflow file path, and the --debug and --workflow options are passed to the CLI. Finally, the status code of the CLI command is stored in the status variable.

In [22]:
#Run the workflow
stdout, stderr, status = Open3.capture3("#{cli} run --debug --workflow #{osw_path}")
status

#<Process::Status: pid 24356 exit 0>

To view the results, set the file path to an HTML report generated by the "openstudio_results" measure and display the contents of the file using IRuby's display method. The argument passed to the display method is the content of the file read using Ruby's File.read method, and the mime type is specified as 'text/html' to indicate that the contents are HTML. This code assumes that the file exists and will raise an error if it cannot be found or read.

In [25]:
file_path = File.expand_path(Dir.pwd + '/run/006_measures/openstudio_results/report.html')

#IRuby.display File.read(file_path), mime: 'text/html'

"C:/Projects/Notebooks/osw_project/Optimization/run/006_measures/openstudio_results/report.html"

or launch in an external browser like chrome, etc

In [26]:
Open3.popen3("start chrome.exe #{file_path}")

[#<IO:fd 4>, #<IO:fd 5>, #<IO:fd 7>, #<Process::Waiter:0x000000000d46f660 run>]